# Human Activity Recognition Using WiFi Signals

## Overview
Human Activity Recognition (HAR) using WiFi signals leverages the unique properties of wireless channel variations to detect different activities.

## Data Format
- **WiFi signal data** is similar to image data in structure, represented in the shape `(channels, height, width)`, but with a different interpretation:
  - `channels` → **channel**
  - `height` → **Time Steps**
  - `width` → **Antenna Pairs (transmitter-receiver combinations)**
- **Labels** represent a predefined set of classes, as is typical in classification tasks.

# Reading Data

In [1]:
# The following code will only execute
# successfully when compression is complete

import kagglehub

# Download latest version
path = kagglehub.dataset_download("alihabibullah/question-2-data")

print("Path to dataset files:", path)

100%|██████████| 204M/204M [00:01<00:00, 114MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/alihabibullah/question-2-data/versions/1


In [2]:
import os
os.listdir(path)

['WiFiSensingDataset.pt']

In [3]:
import torch

# Load the .pt file
data = torch.load(f"{path}/WiFiSensingDataset.pt")


<ipython-input-3-4d2815cbd2c7>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(f"{path}/WiFiSensingDataset.pt")


In [20]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Task 1: Analyze the Dataset ( Stored in `data`)

1. **Determine the number of unique labels** in the dataset.  

2. **Determine the shape of the input data** (number of samples and features).  

3. **Find the maximum value** in the dataset.  

4. **Find the minimum value** in the dataset.  

In [10]:
print(data.keys())

dict_keys(['X_test', 'X_train', 'y_test', 'y_train'])


In [11]:
y_train = data['y_train']
y_test = data['y_test']

In [14]:
X_train = data['X_train']
X_test = data['X_test']

In [12]:
all_labels = torch.cat((y_train, y_test), dim=0)

unique_labels = torch.unique(all_labels)

num_unique_labels = len(unique_labels)
print(f"Number of unique labels: {num_unique_labels}")

Number of unique labels: 7


In [17]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

Shape of X_train: torch.Size([2500, 1, 250, 90])
Shape of X_test: torch.Size([500, 1, 250, 90])


#Find the maximum value in the dataset.

In [18]:
max_value = torch.max(torch.cat((X_train, X_test), dim=0))
print(f"Maximum value in the dataset: {max_value}")

Maximum value in the dataset: 1.0


#Find the minimum value in the dataset.

In [19]:
min_value = torch.min(torch.cat((X_train, X_test), dim=0))
print(f"Minimum value in the dataset: {min_value}")

Minimum value in the dataset: 0.0


# Task 2: Build and Evaluate a Neural Network

1. **Design a Neural Network (Maximum 5 Layers)**  
   Build a compact neural network with no more than 5 layers. Clearly specify the type of each layer (e.g., Dense, Conv2D) and any activation functions used.

2. **Evaluate Your Model**  
   Train your network on the provided dataset and report the evaluation metrics (e.g., accuracy, loss). Discuss the performance of your model and any challenges faced during training.


In [21]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()


        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)


        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)


        self.fc1 = nn.Linear(64 * 62 * 22, 128)
        self.fc2 = nn.Linear(128, 7)

    def forward(self, x):

        x = self.pool1(torch.relu(self.conv1(x)))


        x = self.pool2(torch.relu(self.conv2(x)))


        x = x.view(-1, 64 * 62 * 22)


        x = torch.relu(self.fc1(x))


        x = self.fc2(x)

        return x
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(model)

SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=87296, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=7, bias=True)
)


In [22]:
# Convert labels to tensor
y_train = torch.tensor(y_train, dtype=torch.long)  # Ensure labels are of type long
y_test = torch.tensor(y_test, dtype=torch.long)

# Create datasets from tensors
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

<ipython-input-22-ddb5c37401e7>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.long)  # Ensure labels are of type long
<ipython-input-22-ddb5c37401e7>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test, dtype=torch.long)


In [24]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        running_loss += loss.item()


        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")

Epoch [1/3], Loss: 0.8302, Accuracy: 71.28%
Epoch [2/3], Loss: 0.7090, Accuracy: 74.96%
Epoch [3/3], Loss: 0.6101, Accuracy: 79.44%


Good luck in the exam x)

Prepared by: Ahmed Y. Radwan
